# Using pre-trained word embeddings with CNN

This code is taken from KERAS.IO

The original file:
https://keras.io/examples/nlp/pretrained_word_embeddings/

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Introduction

In this example, we show how to train a text classification model that uses pre-trained
word embeddings.

We'll work with the Newsgroup20 dataset, a set of 20,000 message board messages
belonging to 20 different topic categories.

For the pre-trained word embeddings, we'll use
[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

## Download the Newsgroup20 data

In [ ]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 2s 0us/step


## Let's take a look at the data

In [ ]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['comp.windows.x', 'sci.electronics', 'soc.religion.christian', 'sci.crypt', 'comp.graphics', 'sci.space', 'comp.sys.ibm.pc.hardware', 'rec.sport.baseball', 'misc.forsale', 'talk.politics.mideast', 'rec.motorcycles', 'talk.politics.misc', 'comp.os.ms-windows.misc', 'talk.politics.guns', 'talk.religion.misc', 'alt.atheism', 'rec.autos', 'rec.sport.hockey', 'comp.sys.mac.hardware', 'sci.med']
Number of files in comp.graphics: 1000
Some example filenames: ['38645', '38361', '38486', '38849', '38433']


Here's a example of what one file contains:

In [ ]:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



As you can see, there are header lines that are leaking the file's category, either
explicitly (the first line is literally the category name), or implicitly, e.g. via the
`Organization` filed. Let's get rid of the headers:

In [ ]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

There's actually one category that doesn't have the expected number of files, but the
difference is small enough that the problem remains a balanced classification problem.

In [ ]:
len(samples), len(labels)

(19997, 19997)

## Shuffle and split the data into training & validation sets

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [ ]:
train_samples[:3]

["In article <0P6a3B1w165w@cybernet.cse.fau.edu> jimg@cybernet.cse.fau.edu (Jim Gorycki) writes:\n>\n>A little Bio from _Sun-Sentinel_\n>Torrey, the architect of four consecutive Stanley Cup champions as \n>persident and general manager of the New York Islanders.\n>Throughout his 27 years in the NHL, Bill Torrey's bow ties have become\n>as much of a signature as Andre Agassi's hair.\n>\n>The Panthers will introduce a uniform, insignia, and ticket-price \n>information in early next month.  In the meantime, Huizenga leaves the\n>day-to-day operation in the hands of Torrey and Bob Clarke, the VP and\n>GM.\n>\n\nThe San Jose Sharks and Ottawa Senators are each on their second GM\nalready...I'd be willing to wager that both the Sharks and Senators\nwill probably see their 3rd GM's and perhaps their 4th, before we\nsee the Panthers second.\n\nGerald\n",
 'Approved: christian@aramis.rutgers.edu\n\nIn article <May.14.02.11.19.1993.25177@athos.rutgers.edu> seanna@bnr.ca (Seanna (S.M.) Watson) w

## Create a vocabulary index

Let's use the `TextVectorization` to index the vocabulary found in the dataset.
Later, we'll use the same layer instance to vectorize the samples.

Our layer will only consider the top 20,000 words, and will truncate or pad sequences to
be actually 200 tokens long.

In [ ]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

You can retrieve the computed vocabulary used via `vectorizer.get_vocabulary()`. Let's
print the top 5 words:

In [ ]:
vectorizer.get_vocabulary()[0:16]

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'a',
 'and',
 'in',
 'is',
 'i',
 'that',
 'it',
 'for',
 'you',
 'this',
 'on']

In [ ]:
# size of voc
len(vectorizer.get_vocabulary())

20000

Let's vectorize a test sentence:

In [ ]:
output = vectorizer([["the cat sat on the mat"]] )
output.numpy()[0, :6]

array([   2, 3649, 1765,   15,    2, 6025])

In [ ]:
output.numpy()[0, :16]

As you can see, "the" gets represented as "2". Why not 0, given that "the" was the first
word in the vocabulary? That's because index 0 is reserved for padding and index 1 is
reserved for "out of vocabulary" tokens.

Here's a dict mapping words to their indices:

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

As you can see, we obtain the same encoding as above for our test sentence:

In [ ]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3823, 1745, 15, 2, 7459]

## Load pre-trained word embeddings

Let's download pre-trained GloVe embeddings (a 822M zip file).

You'll need to run the following commands:

```
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
```

The archive contains text-encoded vectors of various sizes: 50-dimensional,
100-dimensional, 200-dimensional, 300-dimensional. We'll use the 100D ones.

Let's make a dict mapping words (strings) to their NumPy vector representation:

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-02-17 01:36:36--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-02-17 01:36:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-02-17 01:36:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
os.listdir()

['.config',
 'glove.6B.300d.txt',
 'glove.6B.200d.txt',
 'glove.6B.100d.txt',
 'glove.6B.50d.txt',
 'glove.6B.zip',
 'sample_data']

In [ ]:
#!head -10 glove.6B.50d.txt
!wc -l glove.6B.100d.txt

400000 glove.6B.100d.txt


In [ ]:
path_to_glove_file ="glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embeddings_index["the"]

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

Now, let's prepare a corresponding embedding matrix that we can use in a Keras
`Embedding` layer. It's a simple NumPy matrix where entry at index `i` is the pre-trained
vector for the word of index `i` in our `vectorizer`'s vocabulary.

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 18005 words (1995 misses)


Next, we load the pre-trained word embeddings matrix into an `Embedding` layer.

Note that we set `trainable=False` so as to keep the embeddings fixed (we don't want to
update them during training).

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## Build the model

A simple 1D convnet with global max pooling and a classifier at the end.

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000200   
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         64128     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, None, 128)        0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, None, 128)        0         
 1D)                                                       

## Train the model

First, convert our list-of-strings data to NumPy arrays of integer indices. The arrays
are right-padded.

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [ ]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((15998, 200), (15998,), (3999, 200), (3999,))

We use categorical crossentropy as our loss since we're doing softmax classification.
Moreover, we use `sparse_categorical_crossentropy` since our labels are integers.

In [ ]:
y_train[:5]

array([10, 15, 18,  5, 15])

In [ ]:
x_train[100]

array([    4,   476,  8412,  8670,     2,   200,   688,     3,    16,
           7,  2638,   195,    23,   261,    33,     9,   118,     2,
        6489,   200,     3,    39,     8,     3,  1453,  8313,    28,
        7949,  2834,  1828,     2,   694,     6,     7,    14,     2,
        2834,   906,    18,    21,  8493,    21,  5036,  5700,     3,
        7619,   214,    20,  2992,    11,    78,   688,     3,    17,
        6833,    12,  3049, 11951,     3,     2,  2573,    21,    90,
          21,    63,   728,     7,    65,   509,     9,   118,     3,
        1541,   761,  8670, 15858,    35,    16,     3,   247,   496,
        1952, 17124,  1384,  2483,  1195,    23,    13,    64,     4,
         109,     5,  6489,   200,    10,     8,     4,  1014,   533,
         138,  1402,  2327,   517,     1,   287, 14699,   173,  6010,
           1,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

hist=model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

In [ ]:
hist.history["loss"]

[2.651577949523926,
 1.9744912385940552,
 1.531005620956421,
 1.2855956554412842,
 1.1128093004226685,
 0.990588903427124,
 0.8745663166046143,
 0.776181161403656,
 0.6794454455375671,
 0.602085530757904,
 0.5314925312995911,
 0.45894214510917664,
 0.39452025294303894,
 0.35732564330101013,
 0.307815819978714,
 0.27414336800575256,
 0.25425970554351807,
 0.22366692125797272,
 0.2113034725189209,
 0.1925293505191803]

## TASKS

A) **Inference**

* So far, we learned how to train a model.  But we do not discuss how to apply it for a new textual instance. We should put the capabilities learned during training to work. We call inference that is applying a machine learning model to a dataset and generating an output or prediction.

So write a function **inference(model, textual_input)** to do inference for the CNN based model trained above "pre-trained word embeddings with CNN"

 

In [3]:
def inference(model, textual_input):
  # your codes here
  return

B) **GRADIO** 

Take a look at gradio and build a demo for your model with a user-friendly web interface so that we can use it. 

https://gradio.app/

In [2]:
# your gradio code goes here